<a href="https://colab.research.google.com/github/mmcastillo/al112248/blob/main/organizar_imagenes_aneurismas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
#------------------------------------------------------------------------------------------------------
#--------------------Código para organizar dataset en carpetas train/val/test--------------------------
#------------------------------------------------------------------------------------------------------
#Necesario para llamar por lotes el conjunto de entrenamiento 
#1. Se cargan los volúmenes de los pacientes caso por caso 
#2. Se reescalan y normalizan para obtener volúmenes de 512x512x128 
#(originalmente hay de 560x560x140 y de 512x512x100 o 1024x1024x160)
#3. Se cambia el tipo de dato de float64 a float32 para las imágenes y a uint8 para las máscaras
#4. Guardo los volúmenes en una carpeta como archivos .npy
#5. Se manda llamar la carpeta y se sortean en nuevas carpetas train/val/test 

In [31]:
#pip install split-folders

In [32]:
pip install pypatchify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
#import splitfolders
from google.colab import drive
drive.mount('/content/drive/');
import nibabel as nib
from scipy import ndimage
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import glob
import pypatchify

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [34]:
def resize_volume(img):
    # Set the desired depth
    desired_depth = 128
    desired_width = 512
    desired_height = 512
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

In [35]:
def normalizar(x):
  mn, mx = x.min(), x.max()
  x_scaled = (x - mn) / (mx - mn)
  return x_scaled

In [41]:
#Carga las imágenes y máscaras nifti, las reescala a 512x512x128 y normaliza y las guarda en dos carpetas (imgs, masks) 
ruta_TOF = '/content/drive/MyDrive/DOCTORADO/python/adam_subset/train_images/';
lista_archivos_TOF = sorted(glob.glob(ruta_TOF+'*.nii.gz'))
ruta_mascaras = '/content/drive/MyDrive/DOCTORADO/python/adam_subset/train_tags/'; 
lista_archivos_mascaras = sorted(glob.glob(ruta_mascaras+'*.nii.gz'))

#10 aneurismas 

ruta_destino_imgs = '/content/drive/MyDrive/DOCTORADO/python/ADAM_10A_npy/train/imgs/';
ruta_destino_mascs = '/content/drive/MyDrive/DOCTORADO/python/ADAM_10A_npy/train/masks/';

ruta_destino_val_imgs = '/content/drive/MyDrive/DOCTORADO/python/ADAM_10A_npy/val/imgs/';
ruta_destino_val_mascs = '/content/drive/MyDrive/DOCTORADO/python/ADAM_10A_npy/val/masks/';

In [46]:
n=0
for i in range(len(lista_archivos_TOF)):
  if lista_archivos_TOF[i][-8:]=='B.nii.gz':
    data = nib.load(lista_archivos_TOF[i])
    temp = data.get_fdata()
    temp = resize_volume(temp)
    #temp = normalizar(temp)
    #temp = temp.astype(np.float16)
    temp = np.expand_dims(temp, axis=3)
    temp = np.expand_dims(temp, axis=0)    
    temp = pypatchify.patchify_to_batches(temp, (64 ,64, 64, 1), batch_dim=0)
    n = n+1
    for j in range(temp.shape[0]):
      if n==1:
        np.save(ruta_destino_val_imgs + str(n) + 'A'+ str(j+1) + '.npy', temp[j,:,:,:,:])
      else:
        np.save(ruta_destino_imgs + str(n) + 'A'+ str(j+1) + '.npy', temp[j,:,:,:,:])

In [47]:
n=0
for i in range(len(lista_archivos_mascaras)):
  if lista_archivos_mascaras[i][-8:]=='B.nii.gz':
    data = nib.load(lista_archivos_mascaras[i])
    tempm = data.get_fdata()
    tempm = resize_volume(tempm)
    tempm = np.where(tempm>0,1,0)
    #tempm = tempm.astype(np.float16)
    tempm = np.expand_dims(tempm, axis=3)
    tempm = np.expand_dims(tempm, axis=0)
    tempm = pypatchify.patchify_to_batches(tempm, (64 ,64, 64, 1), batch_dim=0)
    n = n+1
    for j in range(tempm.shape[0]):
      if n==1:
        np.save(ruta_destino_val_mascs + str(n) + 'A' + str(j+1) + '.npy', tempm[j,:,:,:,:])
      else:
        np.save(ruta_destino_mascs + str(n) + 'A' + str(j+1) + '.npy', tempm[j,:,:,:,:])

In [ ]:
#Sortea las imágenes y las máscaras en carpetas train/validation/test especificando el porcentaje de cada conjunto
#la semilla se usa para hacerlo replicable    
#input_folder = '/content/drive/MyDrive/DOCTORADO/python/ADAM_10C_10A_npy/'
#splitfolders.ratio(input_folder, output='/content/drive/MyDrive/DOCTORADO/python/ADAM_10C_10A_TVT_npy', seed=1337, ratio=(.8, .1, .1), move=True ,group_prefix=3) 
